# 1. TensorBoard: Embedding Visualization

이번 튜토리얼에서는 MNIST 이미지(784x1 짜리 벡터)를 embedding 시킨 후 시각화를 해보도록 하겠습니다.

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import tensorflow as tf

### Load MNIST dataset
MNIST 데이터셋을 불러옵니다. 

In [2]:
mnist = input_data.read_data_sets(train_dir='mnist', reshape=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


### Define Embedding Variable
먼저 Embedding 시킬 Variable을 정해줘야 합니다.여기서는 MNIST 테스트 이미지 (784x1)를 Embedding 시켜보도록 하겠습니다.

In [3]:
# TODO: define a embedding variable 
embedding_var = tf.Variable(mnist.test.images, name='embedding')

아래는 logs 폴더를 지우고 새로 생성하는 코드와 gpu 환경설정을 하는 코드입니다. 별로 중요한 코드는 아니니 넘어가도록 하겠습니다.

In [4]:
# clear the log directory and recreate it 
log_dir = 'logs/'
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

# gpu configuration 
gpu_config = tf.ConfigProto()
gpu_config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=gpu_config) 

아래는 saver를 정의하고 model을 저장하는 코드입니다. 여기도 그냥 넘어가도록 하겠습니다.

In [5]:
tf.global_variables_initializer().run()
saver = tf.train.Saver()
saver.save(sess, save_path=log_dir+'model.ckpt', global_step=None)

'logs/model.ckpt'

### Add Embedding
Embedding 시킬 tensor variable을 추가해줍니다.

In [6]:
# TODO: add embedding data 
summary_writer = tf.summary.FileWriter(logdir=log_dir, graph=tf.get_default_graph())
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

### Add Metadata

MNIST 이미지의 meta data로 label과 sprite image를 넣어보도록 하겠습니다.  sprite image는 폴더들을 찾아보면 png파일로 있습니다.

In [7]:
# TODO: add label data 
metadata_path = 'logs/metadata.tsv'
with open(metadata_path, 'w') as f:
    for i, label in enumerate(mnist.test.labels):
        f.write('{}\n'.format(label))
embedding.metadata_path = metadata_path

In [8]:
# TODO: add sprite image
embedding.sprite.image_path = 'mnist/mnist_10k_sprite.png'
embedding.sprite.single_image_dim.extend([28, 28])

### Visualize Embeddings

In [9]:
# TODO: visualize embedding projector
projector.visualize_embeddings(summary_writer, config)

새로운 linux terminal을 키시고 아래처럼 명령어를 입력합니다. 그 다음 웹 브라우저에 `163.152.20.66:6006`를 입력하여 tensorboard를 실행하면 embedding 결과를 확인할 수 있습니다.

```bash
source anaconda2/bin/activate ~/anaconda2
cd davian-tensorflow/notebooks/week5/
tensorboard --logdir=logs --port=6005
```

### Embedding Result
아래는 PCA를 사용하여 mnist 이미지자체를 embedding 했을 때의 모습입니다. 

![alt text](jpg/pca.jpg)